# **Prompts Template**


A prompt template is a structured framework that allows for dynamic generation of prompts based on predefined patterns and placeholders. It typically includes fixed text and variables that can be filled with specific values at runtime. Prompt templates are useful when generating multiple prompts with similar structures but varying content or style.




Lets see an usecase how you can define a prompt template where you can configure values as placeholders and execute and pass it as a input to LLM



In [ ]:
%%capture
# update or install the necessary libraries
%pip install --upgrade langchain_classic langchain_community langchain_aws python-dotenv

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION")

In [3]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement?
"""

response = llm.invoke(prompt)
response.content

' The statement is incorrect because Monday is not followed by Wednesday. Monday is followed by Tuesday, and then Wednesday comes after Tuesday.'

## **Prompt template**

An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.

* **Template String**: A template string is defined, which includes placeholders for `style` and `text`. This template will be used to create dynamic prompts.
* **ChatPromptTemplate**: The ChatPromptTemplate class from LangChain is used to create a prompt template from the defined string.
* **Input Variables**: The input_variables attribute shows the placeholders that need to be filled when formatting the template.

In [5]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:

instructions_var = "you are a helpful it support chatbot assistant for a xyz company"
context_var = "Answer only the internet connectivity related queries"
input_var = "My internet is not working since morning. Please help."
output_var= "provide steps to troubleshoot internet connectivity issues. in a polite tone"

instructions_var1 = "you are a helpful it support chatbot assistant for a xyz company"
context_var1 = "Answer only the firewall related queries"
output_var1= "provide steps to troubleshoot internet connectivity issues. in a polite tone"

template_string1 = """
instructions: {instructions_temp},
context: {context_temp},
input: {input_temp},
output: {output_temp}"""

from langchain_core.prompts import PromptTemplate
prompt_template1 = PromptTemplate.from_template(template_string1)
formatted_prompt1 = prompt_template1.format(
    instructions_temp=instructions_var,
    context_temp=context_var,
    input_temp=input_var,
    output_temp=output_var
)


print(formatted_prompt1)


instructions: you are a helpful it support chatbot assistant for a xyz company,
context: Answer only the internet connectivity related queries,
input: My internet is not working since morning. Please help.,
output: provide steps to troubleshoot internet connectivity issues. in a polite tone


In [21]:
prompt_template1

PromptTemplate(input_variables=['context', 'input', 'instructions', 'output'], input_types={}, partial_variables={}, template='\ninstructions: {instructions},\ncontext: {context},\ninput: {input},\noutput: {output}')

In [22]:
response1 = llm.invoke(formatted_prompt1)

In [23]:
response1.content

" I'm sorry to hear that you're experiencing issues with your internet connection this morning. I'd be happy to help you troubleshoot the problem. Here are some steps you can try to get your internet connection back up and running:\n\n1. Check your internet connection cables: Make sure all the cables connecting your computer to your modem or router are securely plugged in.\n2. Restart your modem and router: Unplug both your modem and router from the power outlet, wait for about 30 seconds, and then plug them back in. Your devices may take a few minutes to reconnect to the network.\n3. Check for network outages: Go to the XYZ company's website or call the support line to see if there's an outage in your area.\n4. Check your firewall settings: Make sure your firewall settings are not blocking your internet connection.\n5. Run the network troubleshooter: If you're using a Windows computer, you can run the network troubleshooter to help diagnose and fix any connection issues.\n6. Disable I

In [6]:
# prompt template
from langchain_classic.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [7]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [8]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

## **Formatting the Employee's Email**

* **Employee Style**: A style specification for translating text is defined (British English in a calm and respectful tone).
* **Employee Email**: A sample French email requesting vacation is provided.
* **Format Messages**: The format_messages method of the prompt template is used to fill in the placeholders with the specified style and text.
* Model Response: The formatted messages are sent to the language model for processing, and the response is printed.

In [9]:
employee_style = """British English \
in a calm and respectful tone
"""

In [10]:
# french language letter for vacation request
employee_email = """
Objet : Demande de Congé

Cher [Nom du Responsable],

Je m'appelle John et je travaille au sein de la société XYZ.
J'aimerais solliciter une demande de congé pour partir en vacances.
Serait-il possible de discuter des dates qui conviendraient le mieux pour l'équipe et l'entreprise?

Je vous remercie par avance pour votre compréhension et j'attends votre retour.

Cordialement,
John

"""

In [11]:
employee_messages = prompt_template.format_messages(
                    style=employee_style,
                    text=employee_email)

In [12]:
employee_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is British English in a calm and respectful tone\n. text: ```\nObjet : Demande de Congé\n\nCher [Nom du Responsable],\n\nJe m'appelle John et je travaille au sein de la société XYZ.\nJ'aimerais solliciter une demande de congé pour partir en vacances.\nSerait-il possible de discuter des dates qui conviendraient le mieux pour l'équipe et l'entreprise?\n\nJe vous remercie par avance pour votre compréhension et j'attends votre retour.\n\nCordialement,\nJohn\n\n```\n", additional_kwargs={}, response_metadata={})]

In [13]:
employee_response = llm.invoke(employee_messages)

In [14]:
employee_response.content

" Dear [Responsible's Name],\n\nI hope this message finds you well. I am John, an employee at XYZ Company.\n\nI would like to request some annual leave in order to go on holiday. Could we possibly discuss some dates that would be most suitable for both the team and the business?\n\nI greatly appreciate your understanding and look forward to your response.\n\nBest regards,\nJohn"

## **Formatting the Manager's Reply**

* **Manager Reply**: A sample reply from a manager is provided in English.
* **Manager Style**: The style for translating the manager's reply is defined (a polite tone that speaks in French).
* **Format Messages**: The prompt template is used again to format the manager's reply with the specified style.
* **Model Response**: The formatted messages are sent to the model, and the response is printed.

In [15]:
manager_reply = """
Subject: Re: Demande de Congé

Hi John,

Thank you for reaching out. I've reviewed your request for vacation leave.\
Please provide the specific dates you'd like to take off, so we can ensure proper coverage during your absence.

Looking forward to your response.

Best regards,
[Manager's Name]
"""

In [16]:
manager_style_pirate = """\
a polite tone \
that speaks in French\
"""

In [17]:
manager_messages = prompt_template.format_messages(
    style=manager_style_pirate,
    text=manager_reply)

In [18]:
manager_response = llm.invoke(manager_messages)

In [19]:
manager_response.content

" ```French\nObjet : Réponse à la Demande de congé\n\nChers John,\n\nJe vous remercie de m'en toucher. Je vais examiner votre demande de congé vacances. Si vous pouviez nous préciser les dates précises que vous préférez prendre du repos, cela nous permettra de garantir une couverture adéquate durant votre absence.\n\nNous attendons votre réponse avec impatience.\n\nMeilleurs voeux,\n[Nom du manager]\n```\n\nTranslation in English:\n\n```English\nSubject: Re: Request for Leave\n\nDear John,\n\nThank you for contacting me. I will review your leave request for vacation. If you could specify the exact dates you prefer to take off, it will help us ensure proper coverage during your absence.\n\nWe look forward to your response.\n\nBest regards,\n[Manager's Name]\n```"

# **Let's Do an Activity**

## **Objective**

Practice creating and utilizing a prompt template to generate customized prompts for a language model.

## **Steps**

* Define a Template String
* Instantiate a Prompt Template
* Prepare Variables
* Format Messages
* Interact with a Language Model